In [32]:
import numpy as np
from ipywidgets import widgets
from PIL import Image, ImageDraw
from tqdm.notebook import tqdm

In [33]:
import utils
import wandb

In [34]:
utils.generate_all_trajectories()

100%|██████████| 200/200 [00:56<00:00,  3.52it/s]
200it [00:00, 2759.24it/s]


In [35]:
from experiments import *

In [36]:
def show(filename):
    file = open(filename, "rb")
    image = file.read()
    return widgets.Image(
        value=image,
        format='gif'
    )

In [37]:
def get_tension_color(perc):
    return min(255, int(255 * (1 + perc))), int(255 * (1 - abs(perc))), min(255, int(255 * (1 - perc)))

In [38]:
wandb.init(project="trajectory-visualization")

In [39]:
gifs_count = 10

## Pendulum animation

In [40]:
experiment_name = "pendulum"
data = utils.get_data(experiment_name, count=gifs_count, sample_size=1000)

In [41]:
for idx in tqdm(range(gifs_count)):
    images = []

    width = 512
    center = width // 2
    color_1 = (0, 0, 0)
    color_2 = (255, 255, 255)
    max_radius = int(center * 1.5)
    step = 5

    joint = (center, width // 3)
    L = width // 3
    r = width // 10

    for i in range(0, 1000, step):
        alpha, _ = data[idx][i]
        im = Image.new('RGB', (width, width), color_1)
        draw = ImageDraw.Draw(im)
        ball_center = (joint[0] + L * np.sin(alpha), joint[1] + L * np.cos(alpha))
        draw.line((joint, ball_center))
        draw.ellipse(((ball_center[0] - r, ball_center[1] - r), (ball_center[0] + r, ball_center[1] + r)))
        images.append(im)

    fname = f'gifs/{experiment_name}{idx}.gif'
    images[0].save(fname,
                   save_all=True, append_images=images[1:], optimize=False, duration=30, loop=0)
    wandb.log({f"{experiment_name} animation": wandb.Video(fname, caption=experiment_name)})

  0%|          | 0/10 [00:00<?, ?it/s]

![](gifs/pendulum0.gif)

## Harmonic oscillator animation

In [42]:
experiment_name = "harmonic_oscillator"
data = utils.get_data(experiment_name, count=gifs_count, sample_size=1000)

In [43]:
for idx in tqdm(range(gifs_count)):
    images = []

    width = 512
    center = width // 2
    color_1 = (0, 0, 0)
    color_2 = (255, 255, 255)
    max_radius = int(center * 1.5)
    step = 5

    joint = (center, width // 6)
    L = width // 3
    a = width // 20

    traj_h = data[idx]

    maxX = np.max(traj_h[:,0])

    for i in range(0, 1000, step):
        raw_x, _ = traj_h[i]
        x = raw_x * (width // 4) // maxX

        im = Image.new('RGB', (width, width), color_1)
        draw = ImageDraw.Draw(im)
        box_center = (center, joint[1] + x + L)
        draw.line((joint, box_center), fill=get_tension_color(raw_x / maxX))
        draw.rectangle(((box_center[0] - a, box_center[1] - a), (box_center[0] + a, box_center[1] + a)))
        images.append(im)

    fname = f'gifs/{experiment_name}{idx}.gif'
    images[0].save(fname,
                   save_all=True, append_images=images[1:], optimize=False, duration=30, loop=0)
    wandb.log({f"{experiment_name} animation": wandb.Video(fname, caption=experiment_name)})

  0%|          | 0/10 [00:00<?, ?it/s]

![](gifs/harmonic_oscillator0.gif)

## Double pendulum animation

In [44]:
experiment_name = "double_pendulum"
data = utils.get_data(experiment_name, count=gifs_count, sample_size=1000)

In [45]:
for idx in tqdm(range(gifs_count)):
    images = []

    width = 512
    center = width // 2
    color_1 = (0, 0, 0)
    color_2 = (255, 255, 255)
    step = 1

    joint = (center, width // 10)
    L = width // 3
    r = width // 20

    for i in range(0, 1000, step):
        theta1, theta2, _, _ = data[idx][i]
        im = Image.new('RGB', (width, width), color_1)
        draw = ImageDraw.Draw(im)
        split_center = (joint[0] + L * np.sin(theta1), joint[1] + L * np.cos(theta1))
        ball_center = (split_center[0] + L * np.sin(theta2), split_center[1] + L * np.cos(theta2))
        draw.line((joint, split_center))
        draw.line((split_center, ball_center))
        draw.ellipse(((split_center[0] - r, split_center[1] - r), (split_center[0] + r, split_center[1] + r)))
        draw.ellipse(((ball_center[0] - r, ball_center[1] - r), (ball_center[0] + r, ball_center[1] + r)))
        images.append(im)

    fname = f'gifs/{experiment_name}{idx}.gif'
    images[0].save(fname,
                   save_all=True, append_images=images[1:], optimize=False, duration=30, loop=0)
    wandb.log({f"{experiment_name} animation": wandb.Video(fname, caption=experiment_name)})

  0%|          | 0/10 [00:00<?, ?it/s]

![](gifs/double_pendulum0.gif)

## Coupled oscillator

In [46]:
experiment_name = "coupled_oscillator"
data = utils.get_data(experiment_name, count=gifs_count, sample_size=1000)

In [48]:
for idx in tqdm(range(gifs_count)):
    images = []

    width = 512
    center = width // 2
    color_1 = (0, 0, 0)
    color_2 = (255, 255, 255)
    step = 1

    joint = (center)
    L = width // 3
    a = width // 10
    quarter = width // 4

    traj_co = data[idx]
    xMax = np.max(traj_co[:,0])

    for i in range(0, 1000, step):
        raw_x1, raw_x2, _, _ = traj_co[i]
        x1 = width // 4 + raw_x1 * (width // 4 - a) // xMax
        x2 = width * 3 // 4 + raw_x2 * (width // 4 - a) // xMax

        im = Image.new('RGB', (width, width), color_1)
        draw = ImageDraw.Draw(im)
        draw.line(((0, center + a), (width, center + a)))
        b1_center = (x1, center)
        b2_center = (x2, center)
        draw.line(((0, center), (x1 - a, center)), fill=get_tension_color((x1 - a) / (quarter - a) - 1))
        draw.line(((x1 + a, center), (x2 - a, center)), fill=get_tension_color((x2 - x1 - a * 2) / (center - a * 2) - 1))
        draw.line(((x2 + a, center), (width, center)), fill=get_tension_color((width - x2 - a) / (quarter - a) - 1))
        draw.rectangle(((b1_center[0] - a, b1_center[1] - a), (b1_center[0] + a, b1_center[1] + a)))
        draw.rectangle(((b2_center[0] - a, b2_center[1] - a), (b2_center[0] + a, b2_center[1] + a)))
        images.append(im)

    fname = f'gifs/{experiment_name}{idx}.gif'
    images[0].save(fname,
                   save_all=True, append_images=images[1:], optimize=False, duration=30, loop=0)
    wandb.log({f"{experiment_name} animation": wandb.Video(fname, caption=experiment_name)})

  0%|          | 0/10 [00:00<?, ?it/s]

![](gifs/coupled_oscillator0.gif)

## Kepler problem

In [49]:
experiment_name = "kepler_problem"
data = utils.get_data(experiment_name, count=gifs_count, sample_size=1000)

In [50]:
for idx in tqdm(range(gifs_count)):
    images = []

    width = 512
    center = width // 2
    color_1 = (0, 0, 0)
    color_2 = (255, 255, 255)
    step = 8

    sun = (center, center)
    R = width // 50
    r = width // 50

    traj_kp = data[idx]

    maxX = np.max(np.abs(traj_kp[:,0]))
    maxY = np.max(np.abs(traj_kp[:,1]))
    maxD = max(maxX, maxY)

    for i in range(0, 1000, step):
        raw_x, raw_y, _, _ = traj_kp[i]
        x = center + raw_x * (width // 2 - a) // maxD
        y = center + raw_y * (width // 2 - a) // maxD

        im = Image.new('RGB', (width, width), color_1)
        draw = ImageDraw.Draw(im)
        draw.ellipse(((sun[0] - R, sun[1] - R), (sun[0] + R, sun[1] + R)), width=3)
        draw.ellipse(((x - r, y - r), (x + r, y + r)))
        images.append(im)

    fname = f'gifs/{experiment_name}{idx}.gif'
    images[0].save(fname,
                   save_all=True, append_images=images[1:], optimize=False, duration=30, loop=0)
    wandb.log({f"{experiment_name} animation": wandb.Video(fname, caption=experiment_name)})

  0%|          | 0/10 [00:00<?, ?it/s]

![](gifs/kepler_problem0.gif)

In [51]:
wandb.finish()